In [1]:
import os, json, joblib, numpy as np, pandas as pd
from pathlib import Path
import warnings 
warnings.filterwarnings("ignore")

from scipy.spatial.transform import Rotation as R

from sklearn.model_selection import StratifiedGroupKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.utils.class_weight import compute_class_weight

from tensorflow.keras.utils import Sequence, to_categorical, pad_sequences
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import (
    Input, Conv1D, BatchNormalization, LayerNormalization, Activation, add, MaxPooling1D, Dropout,
    Bidirectional, LSTM, GlobalAveragePooling1D, Dense, Multiply, Reshape,
    Lambda, Concatenate, GRU, GaussianNoise
)
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import backend as K
import tensorflow as tf
import polars as pl

import matplotlib.pyplot as plt

2025-09-02 09:28:43.800486: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-09-02 09:28:41.235839: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756794521.316467  281726 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756794521.340551  281726 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-09-02 09:28:41.560687: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
import keras
print(tf.__version__)
print(keras.__version__)

2.18.0
3.8.0


In [3]:
print(tf.config.list_physical_devices("GPU"))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [4]:
import tensorflow as tf
print(tf.sysconfig.get_build_info())

OrderedDict([('cpu_compiler', '/usr/lib/llvm-18/bin/clang'), ('cuda_compute_capabilities', ['sm_60', 'sm_70', 'sm_80', 'sm_89', 'compute_90']), ('cuda_version', '12.5.1'), ('cudnn_version', '9'), ('is_cuda_build', True), ('is_rocm_build', False), ('is_tensorrt_build', False)])


In [5]:
import tensorflow as tf
print("GPU sayısı:", len(tf.config.list_physical_devices('GPU')))

GPU sayısı: 1


In [6]:
state_num = 25
import random
def seed_everything(seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    tf.experimental.numpy.random.seed(seed)
    os.environ['TF_CUDNN_DETERMINISTIC'] = '1'
    os.environ['TF_DETERMINISTIC_OPS'] = '1'
seed_everything(seed=state_num)

In [7]:
# (Competition metric will only be imported when TRAINing)
TRAIN = True                
DEBUG_GATE = False
                     
RAW_DIR = Path("")
PRETRAINED_DIR = Path("new_model_10_fold")
EXPORT_DIR = Path("10folds_new")
BATCH_SIZE = 64
PAD_PERCENTILE = 95
LR_INIT = 5e-4
WD = 3e-3
MIXUP_ALPHA = 0.4 
EPOCHS = 160
PATIENCE = 40
N_SPLITS = 10
MASKING_PROB = 0.25
GATE_LOSS_WEIGHT = 0.20     # 0.20

print("▶ imports ready · tensorflow", tf.__version__)

▶ imports ready · tensorflow 2.18.0


In [8]:
def remove_gravity_from_acc(acc_data, rot_data):
    acc_values = acc_data[['acc_x', 'acc_y', 'acc_z']].values
    quat_values = rot_data[['rot_x', 'rot_y', 'rot_z', 'rot_w']].values
    linear_accel = np.zeros_like(acc_values)
    gravity_world = np.array([0, 0, 9.81])
    for i in range(len(acc_values)):
        if np.all(np.isnan(quat_values[i])) or np.all(np.isclose(quat_values[i], 0)):
            linear_accel[i, :] = acc_values[i, :]
            continue
        try:
            rotation = R.from_quat(quat_values[i])
            gravity_sensor_frame = rotation.apply(gravity_world, inverse=True)
            linear_accel[i, :] = acc_values[i, :] - gravity_sensor_frame
        except ValueError:
             linear_accel[i, :] = acc_values[i, :]
    return linear_accel

def calculate_angular_velocity_from_quat(rot_data, time_delta=1/200):
    quat_values = rot_data[['rot_x', 'rot_y', 'rot_z', 'rot_w']].values
    angular_vel = np.zeros((len(quat_values), 3))
    for i in range(len(quat_values) - 1):
        q_t, q_t_plus_dt = quat_values[i], quat_values[i+1]
        if np.all(np.isnan(q_t)) or np.all(np.isnan(q_t_plus_dt)): continue
        try:
            rot_t = R.from_quat(q_t)
            rot_t_plus_dt = R.from_quat(q_t_plus_dt)
            delta_rot = rot_t.inv() * rot_t_plus_dt
            angular_vel[i, :] = delta_rot.as_rotvec() / time_delta
        except ValueError: pass
    return angular_vel

def calculate_angular_distance(rot_data):
    quat_values = rot_data[['rot_x', 'rot_y', 'rot_z', 'rot_w']].values
    angular_dist = np.zeros(len(quat_values))
    for i in range(len(quat_values) - 1):
        q1, q2 = quat_values[i], quat_values[i+1]
        if np.all(np.isnan(q1)) or np.all(np.isnan(q2)): continue
        try:
            r1, r2 = R.from_quat(q1), R.from_quat(q2)
            relative_rotation = r1.inv() * r2
            angular_dist[i] = np.linalg.norm(relative_rotation.as_rotvec())
        except ValueError: pass
    return angular_dist

In [9]:
# Tensor Manipulations
def time_sum(x): return K.sum(x, axis=1) 
def squeeze_last_axis(x): return tf.squeeze(x, axis=-1)
def expand_last_axis(x): return tf.expand_dims(x, axis=-1)

def se_block(x, reduction=8):
    ch = x.shape[-1]
    se = GlobalAveragePooling1D()(x)
    se = Dense(ch // reduction, activation='relu')(se)
    se = Dense(ch, activation='sigmoid')(se)
    se = Reshape((1, ch))(se)
    return Multiply()([x, se])

def residual_se_cnn_block(x, filters, kernel_size, pool_size=2, drop=0.3, wd=1e-4):
    shortcut = x
    for _ in range(2):
        x = Conv1D(filters, kernel_size, padding='same', use_bias=False,
                   kernel_regularizer=l2(wd))(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
    x = se_block(x)
    if shortcut.shape[-1] != filters:
        shortcut = Conv1D(filters, 1, padding='same', use_bias=False,
                          kernel_regularizer=l2(wd))(shortcut)
        shortcut = BatchNormalization()(shortcut)
    x = add([x, shortcut])
    x = Activation('relu')(x)
    x = MaxPooling1D(pool_size)(x)
    x = Dropout(drop)(x)
    return x

def attention_layer(inputs):
    score = Dense(1, activation='tanh')(inputs)
    score = Lambda(squeeze_last_axis)(score)
    weights = Activation('softmax')(score)
    weights = Lambda(expand_last_axis)(weights)
    context = Multiply()([inputs, weights])
    context = Lambda(time_sum)(context)
    return context

In [10]:
class GatedMixupGenerator(Sequence):
    def __init__(self, X, y, batch_size, imu_dim, class_weight=None, alpha=0.2, masking_prob=0.0):
        self.X, self.y = X, y
        self.batch = batch_size
        self.imu_dim = imu_dim
        self.class_weight = class_weight
        self.alpha = alpha
        self.masking_prob = masking_prob
        self.indices = np.arange(len(X))
        
    def __len__(self):
        return int(np.ceil(len(self.X) / self.batch))

    def __getitem__(self, i):
        idx = self.indices[i*self.batch:(i+1)*self.batch]
        Xb, yb = self.X[idx].copy(), self.y[idx].copy()
        
        
        sample_weights = np.ones(len(Xb), dtype='float32')
        if self.class_weight:
            y_integers = yb.argmax(axis=1)
            sample_weights = np.array([self.class_weight[i] for i in y_integers])
        
        gate_target = np.ones(len(Xb), dtype='float32')
        if self.masking_prob > 0:
            for i in range(len(Xb)):
                if np.random.rand() < self.masking_prob:
                    Xb[i, :, self.imu_dim:] = 0
                    gate_target[i] = 0.0

        if self.alpha > 0:
            lam = np.random.beta(self.alpha, self.alpha)
            perm = np.random.permutation(len(Xb))
            X_mix = lam * Xb + (1 - lam) * Xb[perm]
            y_mix = lam * yb + (1 - lam) * yb[perm]
            gate_target_mix = lam * gate_target + (1 - lam) * gate_target[perm]
            sample_weights_mix = lam * sample_weights + (1 - lam) * sample_weights[perm]
            return X_mix, {'main_output': y_mix, 'tof_gate': gate_target_mix}, sample_weights_mix

        return Xb, {'main_output': yb, 'tof_gate': gate_target}, sample_weights

    def on_epoch_end(self):
        #if self.shuffle:
        np.random.shuffle(self.indices)


def build_gated_two_branch_model(pad_len, imu_dim, tof_dim, n_classes, wd=1e-4):
    inp = Input(shape=(pad_len, imu_dim+tof_dim))
    imu = Lambda(lambda t: t[:, :, :imu_dim])(inp)
    tof = Lambda(lambda t: t[:, :, imu_dim:])(inp)

    x1 = residual_se_cnn_block(imu, 64, 3, drop=0.1, wd=wd)
    x1 = residual_se_cnn_block(x1, 128, 5, drop=0.1, wd=wd)

    x2_base = Conv1D(64, 3, padding='same', use_bias=False, kernel_regularizer=l2(wd))(tof)
    x2_base = LayerNormalization()(x2_base); x2_base = Activation('relu')(x2_base)
    x2_base = MaxPooling1D(2)(x2_base); x2_base = Dropout(0.2)(x2_base)
    x2_base = Conv1D(128, 3, padding='same', use_bias=False, kernel_regularizer=l2(wd))(x2_base)
    x2_base = LayerNormalization()(x2_base); x2_base = Activation('relu')(x2_base)
    x2_base = MaxPooling1D(2)(x2_base); x2_base = Dropout(0.2)(x2_base)
    
    gate_input = GlobalAveragePooling1D()(tof)
    gate_input = Dense(16, activation='relu')(gate_input)
    
    gate = Dense(1, activation='sigmoid', name='tof_gate')(gate_input)
    
    x2 = Multiply()([x2_base, gate])

    merged = Concatenate()([x1, x2])
    xa = Bidirectional(LSTM(128, return_sequences=True, kernel_regularizer=l2(wd)))(merged)
    xb = Bidirectional(GRU(128, return_sequences=True, kernel_regularizer=l2(wd)))(merged)
    xc = GaussianNoise(0.09)(merged)
    xc = Dense(16, activation='elu')(xc)
    x = Concatenate()([xa, xb, xc])
    x = Dropout(0.4)(x)
    x = attention_layer(x)
    for units, drop in [(256, 0.5), (128, 0.3)]:
        x = Dense(units, use_bias=False, kernel_regularizer=l2(wd))(x)
        x = BatchNormalization()(x); x = Activation('relu')(x)
        x = Dropout(drop)(x)
    
    out = Dense(n_classes, activation='softmax', name='main_output', kernel_regularizer=l2(wd))(x)
    
    
    return Model(inputs=inp, outputs=[out, gate])

# TRANSFORMER - (LS'den 0.816LB 0.8359 CV)
import tensorflow as tf
from tensorflow.keras.layers import (
    Input, Dense, Dropout, BatchNormalization, Conv1D, MaxPooling1D,
    Activation, Multiply, Bidirectional, LSTM, GRU, Concatenate,
    GlobalAveragePooling1D, Lambda, RepeatVector, Reshape, add,
    LayerNormalization, MultiHeadAttention, Flatten
)
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2
import tensorflow.keras.backend as K

# -----------------
# SE Block
# -----------------
class SEBlock(tf.keras.layers.Layer):
    def __init__(self, reduction=8, **kwargs):
        super().__init__(**kwargs)
        self.reduction = reduction

    def build(self, input_shape):
        ch = input_shape[-1]
        self.gap = GlobalAveragePooling1D()
        self.fc1 = Dense(ch // self.reduction, activation='relu')
        self.fc2 = Dense(ch, activation='sigmoid')
        self.reshape = Reshape((1, ch))
        self.multiply = Multiply()

    def call(self, x):
        se = self.gap(x)
        se = self.fc1(se)
        se = self.fc2(se)
        se = self.reshape(se)
        return self.multiply([x, se])

# -----------------
# Residual SE CNN Block
# -----------------
class ResidualSEBlock(tf.keras.layers.Layer):
    def __init__(self, filters, kernel_size, pool_size=2, drop=0.3, wd=1e-4, **kwargs):
        super().__init__(**kwargs)
        self.filters = filters
        self.kernel_size = kernel_size
        self.pool_size = pool_size
        self.drop = drop
        self.wd = wd

    def build(self, input_shape):
        self.conv_layers = []
        for _ in range(2):
            self.conv_layers.append(Conv1D(self.filters, self.kernel_size, padding='same',
                                           use_bias=False, kernel_regularizer=l2(self.wd)))
            self.conv_layers.append(BatchNormalization())
            self.conv_layers.append(Activation('relu'))

        self.se_block = SEBlock()

        self.shortcut_conv = None
        if input_shape[-1] != self.filters:
            self.shortcut_conv = Conv1D(self.filters, 1, padding='same',
                                        use_bias=False, kernel_regularizer=l2(self.wd))
            self.shortcut_bn = BatchNormalization()

        self.add_layer = add
        self.relu = Activation('relu')
        self.pool = MaxPooling1D(self.pool_size)
        self.dropout = Dropout(self.drop)

    def call(self, x):
        shortcut = x
        for layer in self.conv_layers:
            x = layer(x)
        x = self.se_block(x)

        if self.shortcut_conv is not None:
            shortcut = self.shortcut_conv(shortcut)
            shortcut = self.shortcut_bn(shortcut)

        x = self.add_layer([x, shortcut])
        x = self.relu(x)
        x = self.pool(x)
        x = self.dropout(x)
        return x

# -----------------
# Attention Layer
# -----------------
class AttentionLayer(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def build(self, input_shape):
        self.score_dense = Dense(1, activation='tanh')
        self.softmax = Activation('softmax')
        self.multiply = Multiply()

    def call(self, inputs):
        score = self.score_dense(inputs)
        score = tf.squeeze(score, axis=-1)
        weights = self.softmax(score)
        weights = tf.expand_dims(weights, axis=-1)
        context = self.multiply([inputs, weights])
        return tf.reduce_sum(context, axis=1)

# -----------------
# Transformer Encoder Block
# -----------------
class TransformerEncoderBlock(tf.keras.layers.Layer):
    def __init__(self, head_num=4, ff_dim=256, dropout=0.2, **kwargs):
        super().__init__(**kwargs)
        self.head_num = head_num
        self.ff_dim = ff_dim
        self.dropout_rate = dropout

    def build(self, input_shape):
        self.ln1 = LayerNormalization(epsilon=1e-6)
        self.attn = MultiHeadAttention(num_heads=self.head_num, key_dim=input_shape[-1] // self.head_num)
        self.drop1 = Dropout(self.dropout_rate)

        self.ln2 = LayerNormalization(epsilon=1e-6)
        self.ffn_dense1 = Dense(self.ff_dim, activation='relu')
        self.ffn_dense2 = Dense(input_shape[-1])
        self.drop2 = Dropout(self.dropout_rate)

    def call(self, inputs):
        x_norm = self.ln1(inputs)
        attn_out = self.attn(x_norm, x_norm)
        attn_out = self.drop1(attn_out)
        out1 = attn_out + inputs

        x_norm2 = self.ln2(out1)
        x_ff = self.ffn_dense1(x_norm2)
        x_ff = self.ffn_dense2(x_ff)
        x_ff = self.drop2(x_ff)
        return x_ff + out1

# -----------------
# Model Builder
# -----------------
def build_competition_model(pad_len, imu_dim, tof_dim, n_classes, wd=1e-4):
    inp = Input(shape=(pad_len, imu_dim + tof_dim), name='input_all')

    imu = Lambda(lambda t: t[:, :, :imu_dim], name='imu_slice')(inp)
    tof = Lambda(lambda t: t[:, :, imu_dim:], name='tof_slice')(inp)

    x_imu = ResidualSEBlock(64, 3, pool_size=2, drop=0.1, wd=wd)(imu)
    x_imu = ResidualSEBlock(128, 5, pool_size=2, drop=0.1, wd=wd)(x_imu)

    x_tof = Conv1D(64, 3, padding='same', kernel_regularizer=l2(wd), use_bias=False)(tof)
    x_tof = BatchNormalization()(x_tof)
    x_tof = Activation('relu')(x_tof)
    x_tof = MaxPooling1D(2)(x_tof)

    x_tof = Conv1D(128, 3, padding='same', kernel_regularizer=l2(wd), use_bias=False)(x_tof)
    x_tof = BatchNormalization()(x_tof)
    x_tof = Activation('relu')(x_tof)
    x_tof = MaxPooling1D(2)(x_tof)

    x_tof = Bidirectional(GRU(64, return_sequences=True, kernel_regularizer=l2(wd)))(x_tof)
    x_tof = Dropout(0.2)(x_tof)

    gate_input = GlobalAveragePooling1D()(tof)
    gate_dense = Dense(16, activation='relu')(gate_input)
    gate = Dense(1, activation='sigmoid', name='tof_gate')(gate_dense)

    gate_timesteps = K.int_shape(x_tof)[1]
    gate_expanded = RepeatVector(gate_timesteps)(gate)
    x_tof = Multiply(name='gated_tof_output')([x_tof, gate_expanded])

    merged = Concatenate(name='merged_features')([x_imu, x_tof])

    #x_lstm = Bidirectional(LSTM(128, return_sequences=True, kernel_regularizer=l2(wd)))(merged)
    #x_gru = Bidirectional(GRU(128, return_sequences=True, kernel_regularizer=l2(wd)))(merged)
    x_lstm = Bidirectional(LSTM(160, return_sequences=True, kernel_regularizer=l2(wd)))(merged)
    x_gru = Bidirectional(GRU(160, return_sequences=True, kernel_regularizer=l2(wd)))(merged)
    x_rnn_concat = Concatenate()([x_lstm, x_gru])

    #x_trans = TransformerEncoderBlock(head_num=8, ff_dim=512, dropout=0.3)(x_rnn_concat)
    x_trans = TransformerEncoderBlock(head_num=12, ff_dim=768, dropout=0.3)(x_rnn_concat)
    x_trans = TransformerEncoderBlock(head_num=8, ff_dim=512, dropout=0.3)(x_trans)

    attn_out = AttentionLayer()(x_trans)

    x = attn_out
    for units, drop_rate in [(256, 0.5), (128, 0.3)]:
        x = Dense(units, kernel_regularizer=l2(wd), use_bias=False)(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = Dropout(drop_rate)(x)

    out = Dense(n_classes, activation='softmax', name='main_output', kernel_regularizer=l2(wd))(x)

    return Model(inputs=inp, outputs=[out, gate])


# TRANSFORMER OPTIMIZE (Lambda fix for H5) 0.8429 CV - 0.826 LB -------------- BEST TRANSFORMER
import tensorflow as tf
from tensorflow.keras.layers import (
    Input,
    Dense,
    Dropout,
    BatchNormalization,
    Conv1D,
    MaxPooling1D,
    Activation,
    Multiply,
    Bidirectional,
    LSTM,
    GRU,
    Concatenate,
    GlobalAveragePooling1D,
    Lambda,
    RepeatVector,
    Reshape,
    add,
    LayerNormalization,
    MultiHeadAttention
)
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2

# -----------------
# SE Block
# -----------------
class SEBlock(tf.keras.layers.Layer):
    def __init__(self, reduction=8, **kwargs):
        super().__init__(**kwargs)
        self.reduction = reduction

    def build(self, input_shape):
        ch = int(input_shape[-1])
        self.gap = GlobalAveragePooling1D()
        self.fc1 = Dense(max(ch // self.reduction, 4), activation='relu')
        self.fc2 = Dense(ch, activation='sigmoid')
        self.reshape = Reshape((1, ch))

    def call(self, x):
        se = self.gap(x)
        se = self.fc1(se)
        se = self.fc2(se)
        se = self.reshape(se)
        return x * se

# -----------------
# Residual SE Block (with pooling)
# -----------------
class ResidualSEBlock(tf.keras.layers.Layer):
    def __init__(self, filters, kernel_size, pool_size=2, drop=0.25, wd=1e-4, **kwargs):
        super().__init__(**kwargs)
        self.filters = filters
        self.kernel_size = kernel_size
        self.pool_size = pool_size
        self.drop = drop
        self.wd = wd

    def build(self, input_shape):
        self.conv1 = Conv1D(self.filters, self.kernel_size, padding='same', use_bias=False, kernel_regularizer=l2(self.wd))
        self.bn1 = BatchNormalization()
        self.act1 = Activation('relu')
        self.conv2 = Conv1D(self.filters, self.kernel_size, padding='same', use_bias=False, kernel_regularizer=l2(self.wd))
        self.bn2 = BatchNormalization()
        self.act2 = Activation('relu')
        self.se = SEBlock(reduction=8)
        self.shortcut_conv = None
        if int(input_shape[-1]) != self.filters:
            self.shortcut_conv = Conv1D(self.filters, 1, padding='same', use_bias=False, kernel_regularizer=l2(self.wd))
            self.shortcut_bn = BatchNormalization()
        self.pool = MaxPooling1D(self.pool_size)
        self.dropout = Dropout(self.drop)
        self.add = add
        self.relu = Activation('relu')

    def call(self, x, training=False):
        shortcut = x
        x = self.conv1(x)
        x = self.bn1(x, training=training)
        x = self.act1(x)
        x = self.conv2(x)
        x = self.bn2(x, training=training)
        x = self.act2(x)
        x = self.se(x)
        if self.shortcut_conv is not None:
            shortcut = self.shortcut_conv(shortcut)
            shortcut = self.shortcut_bn(shortcut, training=training)
        x = self.add([x, shortcut])
        x = self.relu(x)
        x = self.pool(x)
        x = self.dropout(x, training=training)
        return x

# -----------------
# Attention pooling (time-wise)
# -----------------
class AttentionLayer(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def build(self, input_shape):
        self.score_dense = Dense(1, activation='tanh')
        self.softmax = tf.keras.layers.Softmax(axis=1)

    def call(self, inputs):
        score = self.score_dense(inputs)  # (B, T, 1)
        score = tf.squeeze(score, axis=-1)  # (B, T)
        weights = self.softmax(score)  # (B, T)
        weights = tf.expand_dims(weights, axis=-1)  # (B, T, 1)
        context = inputs * weights  # (B, T, C)
        return tf.reduce_sum(context, axis=1)  # (B, C)

# -----------------
# Transformer Encoder (pre-LN)
# -----------------
class TransformerEncoderBlock(tf.keras.layers.Layer):
    def __init__(self, head_num=8, ff_dim=512, dropout=0.2, **kwargs):
        super().__init__(**kwargs)
        self.head_num = head_num
        self.ff_dim = ff_dim
        self.dropout_rate = dropout

    def build(self, input_shape):
        d_model = int(input_shape[-1])
        self.ln1 = LayerNormalization(epsilon=1e-6)
        # key_dim must be >=1 and typically d_model // head_num
        self.mha = MultiHeadAttention(num_heads=self.head_num, key_dim=max(1, d_model // self.head_num))
        self.dropout1 = Dropout(self.dropout_rate)
        self.ln2 = LayerNormalization(epsilon=1e-6)
        self.ffn_dense1 = Dense(self.ff_dim, activation='relu', kernel_regularizer=l2(1e-4))
        self.ffn_dense2 = Dense(d_model, kernel_regularizer=l2(1e-4))
        self.dropout2 = Dropout(self.dropout_rate)

    def call(self, x, training=False):
        # Pre-LN
        x_norm = self.ln1(x)
        attn_out = self.mha(x_norm, x_norm)
        attn_out = self.dropout1(attn_out, training=training)
        x = attn_out + x
        x_norm2 = self.ln2(x)
        x_ff = self.ffn_dense1(x_norm2)
        x_ff = self.ffn_dense2(x_ff)
        x_ff = self.dropout2(x_ff, training=training)
        return x + x_ff

# -----------------
# Model builder (Lambda-safe)
# -----------------
def build_competition_model(pad_len, imu_dim, tof_dim, n_classes, wd=1e-4):
    inp = Input(shape=(pad_len, imu_dim + tof_dim), name='input_all')

    # Dilimleme Lambda'larına output_shape verildi (H5 yüklemesi güvenli)
    imu = Lambda(lambda t: t[:, :, :imu_dim], output_shape=(pad_len, imu_dim), name='imu_slice')(inp)
    tof = Lambda(lambda t: t[:, :, imu_dim:], output_shape=(pad_len, tof_dim), name='tof_slice')(inp)

    # IMU path
    x_imu = ResidualSEBlock(64, 3, pool_size=2, drop=0.12, wd=wd)(imu)
    x_imu = ResidualSEBlock(128, 5, pool_size=2, drop=0.12, wd=wd)(x_imu)

    # ToF path
    x_tof = Conv1D(64, 3, padding='same', kernel_regularizer=l2(wd), use_bias=False)(tof)
    x_tof = BatchNormalization()(x_tof)
    x_tof = Activation('relu')(x_tof)
    x_tof = MaxPooling1D(2)(x_tof)
    x_tof = Conv1D(128, 3, padding='same', kernel_regularizer=l2(wd), use_bias=False)(x_tof)
    x_tof = BatchNormalization()(x_tof)
    x_tof = Activation('relu')(x_tof)
    x_tof = MaxPooling1D(2)(x_tof)
    x_tof = Bidirectional(GRU(64, return_sequences=True, kernel_regularizer=l2(wd)))(x_tof)
    x_tof = Dropout(0.22)(x_tof)

    # --- Channel-wise ToF gate (Lambda -> RepeatVector ile değiştirildi) ---
    gate_feat = GlobalAveragePooling1D()(x_tof)  # (B, C)
    gate_chan = Dense(int(x_tof.shape[-1]), activation='sigmoid', name='tof_gate_chan')(gate_feat)  # (B, C)
    gate_chan_expand = RepeatVector(1, name='tof_gate_chan_expand')(gate_chan)  # (B, 1, C) — shape belirgin
    x_tof = Multiply(name='tof_channel_gate')([x_tof, gate_chan_expand])  # (B, T, C) * (B, 1, C)

    # Scalar aux gate (aynı kaldı)
    gate_scalar = Dense(1, activation='sigmoid', name='tof_gate')(gate_feat)  # (B, 1)

    # Merge
    merged = Concatenate(name='merged_features')([x_imu, x_tof])

    # RNN katmanı
    x_lstm = Bidirectional(LSTM(160, return_sequences=True, kernel_regularizer=l2(wd)))(merged)
    x_gru = Bidirectional(GRU(160, return_sequences=True, kernel_regularizer=l2(wd)))(merged)
    x_rnn_concat = Concatenate(name='rnn_concat')([x_lstm, x_gru])

    # Transformer stack
    x_trans = TransformerEncoderBlock(head_num=12, ff_dim=768, dropout=0.28)(x_rnn_concat)
    x_trans = TransformerEncoderBlock(head_num=8, ff_dim=512, dropout=0.28)(x_trans)

    # Temporal attention pooling
    attn_out = AttentionLayer()(x_trans)

    # Dense head
    x = tf.keras.layers.LayerNormalization(epsilon=1e-6)(attn_out)
    for units, drop_rate in [(256, 0.48), (128, 0.32)]:
        x = Dense(units, kernel_regularizer=l2(wd), use_bias=False)(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = Dropout(drop_rate)(x)

    out = Dense(n_classes, activation='softmax', name='main_output', kernel_regularizer=l2(wd))(x)

    return Model(inputs=inp, outputs=[out, gate_scalar])



def build_gated_two_branch_model(pad_len, imu_dim, tof_dim, n_classes, wd=1e-4):
    inp = Input(shape=(pad_len, imu_dim+tof_dim))
    imu = Lambda(lambda t: t[:, :, :imu_dim])(inp)
    tof = Lambda(lambda t: t[:, :, imu_dim:])(inp)

    x1 = residual_se_cnn_block(imu, 64, 3, drop=0.1, wd=wd)
    x1 = residual_se_cnn_block(x1, 128, 5, drop=0.1, wd=wd)

    x2_base = Conv1D(64, 3, padding='same', use_bias=False, kernel_regularizer=l2(wd))(tof)
    x2_base = BatchNormalization()(x2_base); x2_base = Activation('relu')(x2_base)
    x2_base = MaxPooling1D(2)(x2_base); x2_base = Dropout(0.2)(x2_base)
    x2_base = Conv1D(128, 3, padding='same', use_bias=False, kernel_regularizer=l2(wd))(x2_base)
    x2_base = BatchNormalization()(x2_base); x2_base = Activation('relu')(x2_base)
    x2_base = MaxPooling1D(2)(x2_base); x2_base = Dropout(0.2)(x2_base)
    
    gate_input = GlobalAveragePooling1D()(tof)
    gate_input = Dense(16, activation='relu')(gate_input)
    
    gate = Dense(1, activation='sigmoid', name='tof_gate')(gate_input)
    
    x2 = Multiply()([x2_base, gate])

    merged = Concatenate()([x1, x2])
    xa = Bidirectional(LSTM(128, return_sequences=True, kernel_regularizer=l2(wd)))(merged)
    xb = Bidirectional(GRU(128, return_sequences=True, kernel_regularizer=l2(wd)))(merged)
    xc = GaussianNoise(0.09)(merged)
    xc = Dense(16, activation='elu')(xc)
    x = Concatenate()([xa, xb, xc])
    x = Dropout(0.4)(x)
    x = attention_layer(x)
    for units, drop in [(256, 0.5), (128, 0.3)]:
        x = Dense(units, use_bias=False, kernel_regularizer=l2(wd))(x)
        x = BatchNormalization()(x); x = Activation('relu')(x)
        x = Dropout(drop)(x)
    
    out = Dense(n_classes, activation='softmax', name='main_output', kernel_regularizer=l2(wd))(x)
    
    
    return Model(inputs=inp, outputs=[out, gate])

In [11]:
# NEW 0.825
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2
import tensorflow.keras.backend as K

def build_gated_two_branch_model(pad_len, imu_dim, tof_dim, n_classes, wd=1e-4):
    inp = Input(shape=(pad_len, imu_dim + tof_dim))

    # --- Branch split ---
    imu = Lambda(lambda t: t[:, :, :imu_dim], name='imu_input_slice')(inp)
    tof = Lambda(lambda t: t[:, :, imu_dim:], name='tof_input_slice')(inp)

    # --- IMU branch (residual_se_cnn_block'ların zaman boyutunu koruduğunu varsayarak) ---
    # Eğer bu bloklar downsampling yapmıyorsa, x1'in boyutu (None, pad_len, 128) civarında olacaktır.
    # Ancak hata mesajı (None, 31, 128) olduğuna göre, IMU dalının zaten downsampled olduğu varsayılıyor.
    # Bu durumda, x1'in zaman boyutu 31 olarak kabul ediliyor.
    x1 = residual_se_cnn_block(imu, 64, 3, drop=0.1, wd=wd)
    x1 = residual_se_cnn_block(x1, 128, 5, drop=0.1, wd=wd)
    # Hata mesajına göre (None, 31, 128) şekline ulaştığı varsayılıyor.
    # Eğer residual_se_cnn_block iç yapısında MaxPooling yoksa, bu satıra bir MaxPooling1D(2) veya (4) eklemek gerekebilir.

    # --- ToF branch: CNN + BiGRU ---
    x2 = Conv1D(64, 3, padding='same', use_bias=False, kernel_regularizer=l2(wd))(tof)
    x2 = BatchNormalization()(x2)
    x2 = Activation('relu')(x2)
    x2 = MaxPooling1D(2, name='tof_pool_1')(x2) # İlk MaxPooling: (None, pad_len/2, 64)
    x2 = Dropout(0.2)(x2)

    # Hata mesajı (None, 63, 128) olduğuna göre, x2'nin şu anki boyutu 63.
    # x1'in boyutu 31 olduğuna göre, x2'yi de 31'e düşürmek için BİR KEZ DAHA MaxPooling uygulamalıyız.
    # Önceki modelinizde iki kez MaxPooling kullanılıyordu, bu da uyumluluğu artırır.
    x2 = Conv1D(128, 3, padding='same', use_bias=False, kernel_regularizer=l2(wd))(x2) # İkinci Conv1D
    x2 = BatchNormalization()(x2)
    x2 = Activation('relu')(x2)
    x2 = MaxPooling1D(2, name='tof_pool_2')(x2) # İkinci MaxPooling: (None, pad_len/4, 128)
    x2 = Dropout(0.2)(x2)

    x2 = Bidirectional(GRU(64, return_sequences=True, kernel_regularizer=l2(wd)))(x2)
    x2 = Dropout(0.2)(x2)
    # GRU katmanı çıktı boyutunu (None, pad_len/4, 128) veya (None, pad_len/4, 2*64) yapar, yani 128.

    # --- Gate mekanizması korunuyor ---
    # Gate hala orijinal ToF verisinden beslenmeli (downsampling yapılmamış `tof` girdisinden).
    gate_input = GlobalAveragePooling1D()(tof)
    gate_input = Dense(16, activation='relu')(gate_input)
    gate = Dense(1, activation='sigmoid', name='tof_gate')(gate_input)
    
    # Gate'i Multiply katmanında kullanmak için x2'nin mevcut zaman boyutuna göre RepeatVector ile genişletiyoruz.
    # K.int_shape(x2)[1] ile x2'nin dinamik zaman boyutunu alıyoruz.
    current_tof_timesteps = K.int_shape(x2)[1] # Burası şimdi 31 (pad_len / 4) olmalı
    gate_expanded = RepeatVector(current_tof_timesteps)(gate) # (None, current_tof_timesteps, 1)
    
    x2 = Multiply(name='gated_tof_output')([x2, gate_expanded]) # Gate, x2'nin her zaman adımına uygulanır  
    
    # --- Merge ---
    # Şimdi x1 ve x2'nin zaman boyutları (None, 31, features) şeklinde olmalı
    merged = Concatenate(name='merged_branches')([x1, x2])

    # --- Temporal Encoder (GRU + LSTM) + Transformer block mantığı ---
    xa = Bidirectional(LSTM(128, return_sequences=True, kernel_regularizer=l2(wd)))(merged)
    xb = Bidirectional(GRU(128, return_sequences=True, kernel_regularizer=l2(wd)))(merged)

    # --- Lightweight Transformer-style attention block ---
    attn_input = Concatenate(name='attention_input')([xa, xb])
    attn_output = attention_layer(attn_input)  # Global attention pooling (output: (None, features))

    # --- Dense layers ---
    x = attn_output # `attention_layer` çıktısı (batch_size, features) olduğu için artık zaman boyutu yok.
    for units, drop in [(256, 0.5), (128, 0.3)]:
        x = Dense(units, use_bias=False, kernel_regularizer=l2(wd))(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = Dropout(drop)(x)

    out = Dense(n_classes, activation='softmax', name='main_output', kernel_regularizer=l2(wd))(x)
    
    return Model(inputs=inp, outputs=[out, gate])

In [12]:
from scipy.ndimage import sobel

# ToF için spatial gradyan (sobel) temelli özellikler
def calculate_spatial_tof_features(seq_df, sensor_id):
    # 1D 64-pikseli 8x8'e reshape edip sobel gradyanı alacağız
    pixel_cols = [f"tof_{sensor_id}_v{p}" for p in range(64)]
    tof_data = seq_df[pixel_cols].replace(-1, np.nan).ffill().bfill().fillna(0).values
    
    # Frame sayısı x 64 → (N x 8 x 8)
    N = len(seq_df)
    reshaped = tof_data.reshape(N, 8, 8)
    
    # Spatial gradyanları hesapla (sobel x ve y)
    sobel_x = sobel(reshaped, axis=1)
    sobel_y = sobel(reshaped, axis=2)
    grad_mag = np.sqrt(sobel_x ** 2 + sobel_y ** 2)

    # Özet istatistikleri hesapla
    grad_mean = grad_mag.mean(axis=(1, 2))
    grad_std  = grad_mag.std(axis=(1, 2))
    grad_max  = grad_mag.max(axis=(1, 2))
    
    return pd.DataFrame({
        f'tof_{sensor_id}_grad_mean': grad_mean,
        f'tof_{sensor_id}_grad_std': grad_std,
        f'tof_{sensor_id}_grad_max': grad_max
    }, index=seq_df.index)

In [13]:
from scipy.signal import find_peaks

def count_peaks(series):
    peaks, _ = find_peaks(series, height=np.mean(series))
    return len(peaks)

In [14]:
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

if TRAIN:
    print("▶ TRAIN MODE – loading dataset ...")
    df = pd.read_csv(RAW_DIR / "train.csv")
    
    train_dem_df = pd.read_csv(RAW_DIR / "train_demographics.csv")

    le = LabelEncoder()
    df['gesture_int'] = le.fit_transform(df['gesture'])
    np.save(EXPORT_DIR / "gesture_classes.npy", le.classes_)
    
    acc_y_neg_subjects = (
        df.groupby('subject')['acc_y']
        .mean()
        .loc[lambda x: x < 0]
        .index
        .tolist()
    )
    
    # Bu subject'leri tamamen drop et
    df = df[~df['subject'].isin(acc_y_neg_subjects)].reset_index(drop=True)
    
    # --- [Önemli Değişiklik] Gelişmiş Fiziksel ve İstatistiksel Özellikler ---
    print("  Removing gravity and calculating linear acceleration features...")
    linear_accel_list = [pd.DataFrame(remove_gravity_from_acc(group[['acc_x', 'acc_y', 'acc_z']], group[['rot_x', 'rot_y', 'rot_z', 'rot_w']]), columns=['linear_acc_x', 'linear_acc_y', 'linear_acc_z'], index=group.index) for _, group in df.groupby('sequence_id')]
    df = pd.concat([df, pd.concat(linear_accel_list)], axis=1)
    
    # Lineer İvme Özellikleri
    df['linear_acc_mag'] = np.sqrt(df['linear_acc_x']**2 + df['linear_acc_y']**2 + df['linear_acc_z']**2)
    # df['linear_acc_mag_jerk'] already exists, but consider a smoother derivative or higher order jerks if needed
    df['linear_acc_mag_jerk'] = df.groupby('sequence_id')['linear_acc_mag'].diff().fillna(0) # Keep current for now
    
   
    
    print("  Calculating angular velocity and distance from quaternions...")
    angular_vel_list = [pd.DataFrame(calculate_angular_velocity_from_quat(group[['rot_x', 'rot_y', 'rot_z', 'rot_w']]), columns=['angular_vel_x', 'angular_vel_y', 'angular_vel_z'], index=group.index) for _, group in df.groupby('sequence_id')]
    df = pd.concat([df, pd.concat(angular_vel_list)], axis=1)
    angular_dist_list = [pd.DataFrame(calculate_angular_distance(group[['rot_x', 'rot_y', 'rot_z', 'rot_w']]), columns=['angular_distance'], index=group.index) for _, group in df.groupby('sequence_id')]
    df = pd.concat([df, pd.concat(angular_dist_list)], axis=1)

  
    #- HUSEYİN GUR (GMN) 1.FE 80>81
    # Hız ve İvme için Anlık İstatistiksel Özellikler (Mevcut sensör okumalarına ek olarak) 
    for col in ['acc_x', 'acc_y', 'acc_z',  'linear_acc_x', 'linear_acc_y', 'linear_acc_z', 'angular_vel_x', 'angular_vel_y', 'angular_vel_z']:  # 'rot_w', 'rot_x', 'rot_y', 'rot_z' eksik
        if col in df.columns:
            df[f'{col}_diff'] = df.groupby('sequence_id')[col].diff().fillna(0)
            df[f'{col}_abs_diff'] = np.abs(df.groupby('sequence_id')[col].diff()).fillna(0) # Mutlak fark
    #- HUSEYİN GUR (GMN) 1.FE 80>81

    # --- [Önemli Değişiklik] Fiziksel ve Yeni İstatistiksel FE'yi Yansıtan Özellik Listesi ---
    imu_cols_base = ['acc_x', 'acc_y', 'acc_z'] + [c for c in df.columns if c.startswith('rot_')]  #+ ['handedness', 'height_cm']
    # , 'acc_x_spectral_energy','acc_y_spectral_energy','acc_z_spectral_energy', 'linear_acc_mag_spectral_energy'
    imu_engineered = [
    'linear_acc_mag', 'linear_acc_mag_jerk',
    'angular_vel_x', 'angular_vel_y', 'angular_vel_z', 'angular_distance',
    #'jerk_mad_25'
    #'damj_linear_acc_mag_jerk_mean', 'damj_linear_acc_mag_jerk_std', 'damj_linear_acc_mag_jerk_skew',
    #'damj_jerk_mad_25_mean','damj_jerk_mad_25_std'
   
    ] 
    #- HUSEYİN GUR (GMN) 1.FE 80>81
    # Yeni eklenen differansiyel ve mutlak fark özellikleri
    for col in ['acc_x', 'acc_y', 'acc_z', 'linear_acc_x', 'linear_acc_y', 'linear_acc_z', 'angular_vel_x', 'angular_vel_y', 'angular_vel_z']:
        if col in df.columns:
            imu_engineered.append(f'{col}_diff')
            imu_engineered.append(f'{col}_abs_diff')
     #- HUSEYİN GUR (GMN) 1.FE 80>81

    imu_cols = list(dict.fromkeys(imu_cols_base + imu_engineered))
    
    # HUSEYIN GUR - THM - CGPT 3.FE ÇOK AZ BAŞARISIZ 81>81 FAKAT CV 83
    #df = extract_temporal_thm_features(df)
    #df = extract_spatial_thm_features(df)
    # HUSEYIN GUR - THM - CGPT 3.FE
    
    thm_cols_original = [c for c in df.columns if c.startswith('thm_')]
    
    tof_aggregated_cols_template = []
    for i in range(1, 6): tof_aggregated_cols_template.extend([f'tof_{i}_mean', f'tof_{i}_std', f'tof_{i}_min', f'tof_{i}_max'])

    # Spatial Gradient ToF - HUSEYİN GUR - cgpt 2.FE 81>better 81
    for i in range(1, 6):
        tof_aggregated_cols_template.extend([
            f'tof_{i}_grad_mean', f'tof_{i}_grad_std', f'tof_{i}_grad_max'
        ])
    # Spatial Gradient ToF - HUSEYİN GUR - cgpt 2.FE 81>better 81
    
    final_feature_cols = imu_cols  + tof_aggregated_cols_template + thm_cols_original

    imu_dim_final = len(imu_cols)
    #tof_thm_aggregated_dim_final = len(thm_cols_original) + len(tof_aggregated_cols_template)
    tof_dim = len(tof_aggregated_cols_template)
    thm_dim = len(thm_cols_original)

    print(f"  IMU (phys-based + enhanced) {imu_dim_final} | THM + Aggregated TOF {tof_dim + thm_dim} | total {len(final_feature_cols)} features")
    np.save(EXPORT_DIR / "feature_cols.npy", np.array(final_feature_cols))
    
    print("  Building sequences...")
    seq_gp = df.groupby('sequence_id') 
    X_list_unscaled, y_list_int, groups_list, lens = [], [], [], [] 
    for seq_id, seq_df in seq_gp:
        seq_df_copy = seq_df.copy()
        for i in range(1, 6):
            pixel_cols = [f"tof_{i}_v{p}" for p in range(64)]; tof_data = seq_df_copy[pixel_cols].replace(-1, np.nan)
            seq_df_copy[f'tof_{i}_mean'], seq_df_copy[f'tof_{i}_std'], seq_df_copy[f'tof_{i}_min'], seq_df_copy[f'tof_{i}_max'] = tof_data.mean(axis=1), tof_data.std(axis=1), tof_data.min(axis=1), tof_data.max(axis=1)
            
            # Spatial Gradient ToF - HUSEYİN GUR - cgpt 2.FE 81>better 81
            spatial_feats = calculate_spatial_tof_features(seq_df_copy, i)
            seq_df_copy = pd.concat([seq_df_copy, spatial_feats], axis=1)
            # Spatial Gradient ToF - HUSEYİN GUR - cgpt 2.FE 81>better 81
        
        # Sadece belirlenen nihai özellik sütunlarını kullan
        X_list_unscaled.append(seq_df_copy[final_feature_cols].ffill().bfill().fillna(0).values.astype('float32'))
        y_list_int.append(seq_df_copy['gesture_int'].iloc[0])
        groups_list.append(seq_df_copy['subject'].iloc[0])
        lens.append(len(seq_df_copy))

    print("  Fitting StandardScaler...")
    all_steps_concatenated = np.concatenate(X_list_unscaled, axis=0)
    scaler = StandardScaler().fit(all_steps_concatenated)
    joblib.dump(scaler, EXPORT_DIR / "scaler.pkl")
    
    print("  Scaling and padding sequences...")
    X_scaled_list = [scaler.transform(x_seq) for x_seq in X_list_unscaled]
    pad_len = int(np.percentile(lens, PAD_PERCENTILE)); np.save(EXPORT_DIR / "sequence_maxlen.npy", pad_len)
    X = pad_sequences(X_scaled_list, maxlen=pad_len, padding='post', truncating='post', dtype='float32')
    #y_stratify = np.array(y_list_int)

       # --- DEĞİŞİKLİK BAŞLANGICI --- SOL ELLERİ HER FOLD'A EŞİT DAĞIT!
    subject_acc_x_mean_global = df.groupby('subject')['acc_x'].mean()
    subject_is_acc_x_mean_negative = (subject_acc_x_mean_global < 0).astype(str) # '0' veya '1' string olarak
    
    y_stratify = np.array([f"{gesture_label}_{subject_is_acc_x_mean_negative.loc[sub_id]}"
                           for gesture_label, sub_id in zip(y_list_int, groups_list)])
    # --- DEĞİŞİKLİK SONU ---
    
    groups, y = np.array(groups_list), to_categorical(y_list_int, num_classes=len(le.classes_))
    print("  Starting training with Stratified Group K-Fold CV...")
    sgkf = StratifiedGroupKFold(n_splits=N_SPLITS, shuffle=True, random_state=state_num)
    oof_preds = np.zeros_like(y, dtype='float32')
    
    for fold, (train_idx, val_idx) in enumerate(sgkf.split(X, y_stratify, groups)):
        print(f"\n===== FOLD {fold+1}/{N_SPLITS} =====")
        X_tr, X_val, y_tr, y_val = X[train_idx], X[val_idx], y[train_idx], y[val_idx]
       
        # --- [Önemli Değişiklik] Model Derlemesi ve Geri Çağırmalar ---
        #model = build_gated_two_branch_model(pad_len, imu_dim_final, tof_dim, thm_dim, len(le.classes_), wd=WD)
        model = build_gated_two_branch_model(pad_len, imu_dim_final, tof_dim+thm_dim, len(le.classes_), wd=WD)
        
        # Learning Rate Scheduler ekleme - TEK BAŞINA 80>81
        # Bu scheduler, belirli bir metrik iyileşmediğinde öğrenme oranını azaltır.
        lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_main_output_accuracy',
            mode='max',
            factor=0.5,
            patience=8,
            cooldown=2,
            min_lr=3e-6,
            verbose=1
        )
        
        model.compile(optimizer=Adam(LR_INIT),
                      loss={'main_output': tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
                             'tof_gate': tf.keras.losses.BinaryCrossentropy()
                             },
                      loss_weights={'main_output': 1.0,
                                     'tof_gate': GATE_LOSS_WEIGHT,
                                     },
                      metrics={'main_output': 'accuracy'})
        
        class_weight_dict = dict(enumerate(compute_class_weight('balanced', classes=np.arange(len(le.classes_)), y=y_tr.argmax(1))))
        
        # GatedMixupGenerator'ın imu_dim parametresini güncelledik
        train_gen = GatedMixupGenerator(X_tr, y_tr, batch_size=BATCH_SIZE, imu_dim=imu_dim_final, class_weight=class_weight_dict, alpha=MIXUP_ALPHA)
        val_gen = GatedMixupGenerator(X_val, y_val, batch_size=BATCH_SIZE, imu_dim=imu_dim_final,class_weight=None, alpha=0.0) # İmu_dim burada da doğru olmalı

        #train_gen = GatedMixupGenerator(X_tr, y_tr, batch_size=BATCH_SIZE, imu_dim=imu_dim_final, class_weight=class_weight_dict, alpha=MIXUP_ALPHA, masking_prob=MASKING_PROB)
        #val_gen = GatedMixupGenerator(X_val, y_val, batch_size=BATCH_SIZE, imu_dim=imu_dim_final,class_weight=None, alpha=0.2, masking_prob=0.0) # İmu_dim burada da doğru olmalı

        # EarlyStopping ve LearningRateScheduler'ı birlikte kullan
        cb = [
            EarlyStopping(patience=PATIENCE, restore_best_weights=True, verbose=1, monitor='val_main_output_accuracy', mode='max'),
            lr_scheduler
        ]
        
        model.fit(train_gen, epochs=EPOCHS, validation_data=val_gen, callbacks=cb, verbose=1)
        model.save(EXPORT_DIR / f"gesture_model_fold_{fold}.h5")
        preds_val,_ = model.predict(X_val) # Gate çıktısını ayır
        oof_preds[val_idx] = preds_val

    print("\n✔ Training done.")
    
    # --- [OOF Skoru Hesaplama] ---
    from metric import CompetitionMetric
    true_oof_int = y.argmax(1)
    pred_oof_int = oof_preds.argmax(1)
    
    h_f1_oof = CompetitionMetric().calculate_hierarchical_f1(
        pd.DataFrame({'gesture': le.classes_[true_oof_int]}),
        pd.DataFrame({'gesture': le.classes_[pred_oof_int]}))
    print(f"Overall OOF H‑F1 Score = {h_f1_oof:.4f}")

▶ TRAIN MODE – loading dataset ...
  Removing gravity and calculating linear acceleration features...
  Calculating angular velocity and distance from quaternions...
  IMU (phys-based + enhanced) 31 | THM + Aggregated TOF 40 | total 71 features
  Building sequences...
  Fitting StandardScaler...
  Scaling and padding sequences...
  Starting training with Stratified Group K-Fold CV...

===== FOLD 1/10 =====


I0000 00:00:1756794673.536956  281726 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 21770 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:01:00.0, compute capability: 8.6


Epoch 1/160


I0000 00:00:1756794681.708970  282413 cuda_dnn.cc:529] Loaded cuDNN version 90300


112/112 ━━━━━━━━━━━━━━━━━━━━ 20s 113ms/step - loss: 10.5365 - main_output_accuracy: 0.1453 - main_output_loss: 2.9460 - tof_gate_loss: 0.6921 - val_loss: 8.0983 - val_main_output_accuracy: 0.4451 - val_main_output_loss: 2.3980 - val_tof_gate_loss: 0.5082 - learning_rate: 5.0000e-04
Epoch 2/160
112/112 ━━━━━━━━━━━━━━━━━━━━ 9s 77ms/step - loss: 7.5242 - main_output_accuracy: 0.3704 - main_output_loss: 2.2490 - tof_gate_loss: 0.4948 - val_loss: 6.1467 - val_main_output_accuracy: 0.5462 - val_main_output_loss: 1.9125 - val_tof_gate_loss: 0.3711 - learning_rate: 5.0000e-04
Epoch 3/160
112/112 ━━━━━━━━━━━━━━━━━━━━ 12s 105ms/step - loss: 6.0707 - main_output_accuracy: 0.4618 - main_output_loss: 2.0821 - tof_gate_loss: 0.3650 - val_loss: 5.0113 - val_main_output_accuracy: 0.5709 - val_main_output_loss: 1.6615 - val_tof_gate_loss: 0.2700 - learning_rate: 5.0000e-04
Epoch 4/160
112/112 ━━━━━━━━━━━━━━━━━━━━ 12s 103ms/step - loss: 5.1813 - main_output_accuracy: 0.4826 - main_output_loss: 2.0018 - 

...

24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step

===== FOLD 9/10 =====
Epoch 1/160
112/112 ━━━━━━━━━━━━━━━━━━━━ 18s 116ms/step - loss: 10.4623 - main_output_accuracy: 0.1461 - main_output_loss: 2.8599 - tof_gate_loss: 0.7753 - val_loss: 8.0606 - val_main_output_accuracy: 0.4289 - val_main_output_loss: 2.3730 - val_tof_gate_loss: 0.6072 - learning_rate: 5.0000e-04
Epoch 2/160
112/112 ━━━━━━━━━━━━━━━━━━━━ 12s 109ms/step - loss: 7.5026 - main_output_accuracy: 0.3639 - main_output_loss: 2.2451 - tof_gate_loss: 0.5233 - val_loss: 6.1211 - val_main_output_accuracy: 0.5600 - val_main_output_loss: 1.8838 - val_tof_gate_loss: 0.4280 - learning_rate: 5.0000e-04
Epoch 3/160
112/112 ━━━━━━━━━━━━━━━━━━━━ 13s 113ms/step - loss: 6.0696 - main_output_accuracy: 0.4567 - main_output_loss: 2.0858 - tof_gate_loss: 0.3702 - val_loss: 4.9728 - val_main_output_accuracy: 0.6091 - val_main_output_loss: 1.6231 - val_tof_gate_loss: 0.2990 - learning_rate: 5.0000e-04
Epoch 4/160
112/112 ━━━━━━━━━━━━━━━━━━━━ 12s 109ms/ste

2025-09-02 13:06:13.239014: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_14}}


26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step

===== FOLD 10/10 =====
Epoch 1/160
114/114 ━━━━━━━━━━━━━━━━━━━━ 18s 113ms/step - loss: 10.5137 - main_output_accuracy: 0.1599 - main_output_loss: 2.9014 - tof_gate_loss: 0.4140 - val_loss: 8.2134 - val_main_output_accuracy: 0.3669 - val_main_output_loss: 2.4378 - val_tof_gate_loss: 0.2548 - learning_rate: 5.0000e-04
Epoch 2/160
114/114 ━━━━━━━━━━━━━━━━━━━━ 12s 105ms/step - loss: 7.5766 - main_output_accuracy: 0.3669 - main_output_loss: 2.2201 - tof_gate_loss: 0.2727 - val_loss: 6.3438 - val_main_output_accuracy: 0.4300 - val_main_output_loss: 2.0488 - val_tof_gate_loss: 0.1573 - learning_rate: 5.0000e-04
Epoch 3/160
114/114 ━━━━━━━━━━━━━━━━━━━━ 12s 107ms/step - loss: 6.1138 - main_output_accuracy: 0.4381 - main_output_loss: 2.0655 - tof_gate_loss: 0.1823 - val_loss: 5.2199 - val_main_output_accuracy: 0.4790 - val_main_output_loss: 1.8380 - val_tof_gate_loss: 0.0980 - learning_rate: 5.0000e-04
Epoch 4/160
114/114 ━━━━━━━━━━━━━━━━━━━━ 13s 111ms/st

2025-09-02 13:27:19.785784: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_14}}


23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step

✔ Training done.
Overall OOF H‑F1 Score = 0.8428


In [15]:
def predict(sequence: pl.DataFrame, demographics: pl.DataFrame) -> str:
    df_seq = sequence.to_pandas()
    seq_df_copy = df_seq.copy() 


    linear_accel = remove_gravity_from_acc(df_seq, df_seq)
    df_seq['linear_acc_x'], df_seq['linear_acc_y'], df_seq['linear_acc_z'] = linear_accel[:, 0], linear_accel[:, 1], linear_accel[:, 2]
    df_seq['linear_acc_mag'] = np.sqrt(df_seq['linear_acc_x']**2 + df_seq['linear_acc_y']**2 + df_seq['linear_acc_z']**2)
    df_seq['linear_acc_mag_jerk'] = df_seq['linear_acc_mag'].diff().fillna(0)
    angular_vel = calculate_angular_velocity_from_quat(df_seq)
    df_seq['angular_vel_x'], df_seq['angular_vel_y'], df_seq['angular_vel_z'] = angular_vel[:, 0], angular_vel[:, 1], angular_vel[:, 2]
    df_seq['angular_distance'] = calculate_angular_distance(df_seq)

    for col in ['acc_x', 'acc_y', 'acc_z', 'linear_acc_x', 'linear_acc_y', 'linear_acc_z', 'angular_vel_x', 'angular_vel_y', 'angular_vel_z']:
        if col in df_seq.columns:
            df_seq[f'{col}_diff'] = df_seq.groupby('sequence_id')[col].diff().fillna(0)
            df_seq[f'{col}_abs_diff'] = np.abs(df_seq.groupby('sequence_id')[col].diff()).fillna(0) # Mutlak fark

    for i in range(1, 6):
        pixel_cols = [f"tof_{i}_v{p}" for p in range(64)]; tof_data = df_seq[pixel_cols].replace(-1, np.nan)
        df_seq[f'tof_{i}_mean'], df_seq[f'tof_{i}_std'], df_seq[f'tof_{i}_min'], df_seq[f'tof_{i}_max'] = tof_data.mean(axis=1), tof_data.std(axis=1), tof_data.min(axis=1), tof_data.max(axis=1)
        spatial_feats = calculate_spatial_tof_features(seq_df_copy, i)
        df_seq = pd.concat([df_seq, spatial_feats], axis=1)
        
        
    mat_unscaled = df_seq[final_feature_cols].ffill().bfill().fillna(0).values.astype('float32')
    mat_scaled = scaler.transform(mat_unscaled)
    pad_input = pad_sequences([mat_scaled], maxlen=pad_len, padding='post', truncating='post', dtype='float32')

   
    all_preds = [model.predict(pad_input, verbose=0)[0] for model in models] # 主出力のみ取得
    avg_pred = np.mean(all_preds, axis=0)
    print(str(gesture_classes[avg_pred.argmax()]))
    return str(gesture_classes[avg_pred.argmax()])


In [16]:
if not TRAIN:
    import kaggle_evaluation.cmi_inference_server
    inference_server = kaggle_evaluation.cmi_inference_server.CMIInferenceServer(predict)

    if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
        inference_server.serve()
    else:
        inference_server.run_local_gateway(
            data_paths=(
                '/kaggle/input/cmi-detect-behavior-with-sensor-data/test.csv',
                '/kaggle/input/cmi-detect-behavior-with-sensor-data/test_demographics.csv',
            )
        )